In [24]:
import numpy as np
import dill

#from gensim.models import Word2Vec
from window_based_tagger_config import get_config
from Rpfa import micro_rpfa

import logging
import datetime
import pickle

from CrossValidation import cross_validation
from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from collections import defaultdict
from IterableFP import flatten
from Settings import Settings

CV_FOLDS = 5
DEV_SPLIT = 0.1

settings = Settings()
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
training_pickled = settings.data_directory + "CoralBleaching/Thesis_Dataset/training.pl"

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [45]:
config = get_config(training_folder)
# override this so we don't replace INFREQUENT words
config["min_df"] = 0
tagged_essays2 = load_process_essays(**config)
#config

902 files found
902 essays processed


In [40]:
len(tagged_essays2)

902

In [22]:
for essay in tagged_essays2[0:20]:
    print("*" * 80)
    print(essay.name)
    print("*" * 80)
    sents = []
    for sent in essay.sentences:
        tokens = []
        for i, (wd, tags) in enumerate(sent):
            if i == 0:
                wd = wd[0].upper() + wd[1:]
            tokens.append(wd.strip())
        str_sent = " ".join(tokens).strip()
        sents.append(str_sent)
    str_essay = "\n>>> ".join(sents)
    print(str_essay)

********************************************************************************
EBA1415_AEKD_4_CB_ES-05568.ann
********************************************************************************
What leads to differences in the rates of coral bleaching .
>>> Coral is often mistaken for a rock but it is made up of tiny animals called polyps .
>>> Coral bleaching shows bleaching and healthy bleaching event that makes coral vulnerable to disease and starvation .
>>> Coral bleaching is almost noticeable in the pacific ocean .
********************************************************************************
EBA1415_AEKD_4_CB_ES-05572.ann
********************************************************************************
The part of coral called " zooanthellae " are not getting sunlight .
>>> And if they dont get or much sunlight they start to loses their color .
>>> The reason why is because the " zooanthellae . " if not getting mixed up with carbon dioxide .
>>> The coral also need special water

In [28]:
output_folder = root_folder + "CoReference/Training"
output_folder

'/Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/CoReference/Training'

In [48]:
for essay in tagged_essays2:

    sents = []
    for sent in essay.sentences:
        tokens = []
        for i, (wd, tags) in enumerate(sent):
            if i == 0:
                wd = wd[0].upper() + wd[1:]
            tokens.append(wd.strip())
        str_sent = " ".join(tokens).strip()
        sents.append(str_sent)
    str_essay = "\n".join(sents)
    fname = output_folder + "/" + essay.name.replace(".ann", ".txt")
    #print(fname)
    #continue
    with open(fname, "w+") as f:
        f.write(str_essay)

In [46]:
e = [e for e in tagged_essays2 if e.name.startswith("EBA1415_AEKD_5_CB_ES-05579")][0]
e

In [49]:
# Validate it doesn't replace biology with INFREQUENT
for sent in e.sentences[0:1]:
    print("*" * 80)
    wds = list(zip(*sent))[0]
    print(" ".join(wds))

********************************************************************************
i read over marine biology and has been discovered that there are many different types of corals that live in the ocean that live close to the animals and loses their color .


## NOTES
* Don't replace INFREQUENT's - may break co-reference resolution, and also RNN tagging
* Need to lower case words after processing

In [52]:
((902*7)/60)/60

1.7538888888888888